In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, row_number, abs, count
from pyspark.sql import Window, types

spark_session = SparkSession.builder.enableHiveSupport().master("local[*]").getOrCreate()

graph_path = "/data/sample264"

IllegalArgumentException: u"Error while instantiating 'org.apache.spark.sql.hive.HiveSessionState':"

In [ ]:
def norm(df, key1, key2, field, n): 
    
    window = Window.partitionBy(key1).orderBy(col(field).desc())
        
    tops_df = df.withColumn('row_number', row_number().over(window))\
        .filter(col('row_number') <= n)\
        .drop(col('row_number')) 
        
    tmp_df = tops_df.groupBy(col(key1)).agg(col(key1), sum(col(field)).alias('sum_' + field))
   
    normalized_df = tops_df.join(tmp_df, key1, 'inner')\
        .withColumn('norm_' + field, col(field) / col('sum_' + field))\
        .cache()

    return normalized_df

In [ ]:
graph = spark_session.read.parquet(graph_path)

unnorm = graph\
    .groupBy(col('userId'), col('artistId'))\
    .count()
    
norm = norm(unnorm, 'userId', 'artistId', 'count', 100)\
    .select(col('userId'), col('artistId'))\
    .orderBy(col('norm_count').desc(), col('userId'), col('artistId'))\
    .limit(40)
    
result = norm.collect()

for user_id, artist_id in result:
    print('%s\t%s' % (user_id, artist_id))